In [139]:
# Imports
import numpy as np
import math
import time
import json
import pandas as pd
import requests
from pandas import json_normalize
import os
import csv
import matplotlib.pyplot as plt
from pywaffle import Waffle 

In [20]:
def read_df(i):
    df = pd.read_csv("Season"+str(i)+".csv",encoding = 'utf-8-sig',low_memory=False)
    return df

In [29]:
playerlist = ['Lionel Andrés Messi Cuccittini','David Villa Sánchez','Pedro Eliezer Rodríguez Ledesma']
def plist(df):
    playerlist = df.query("(team_name=='Barcelona')").player_name.dropna().unique().tolist()
    return playerlist

In [120]:
season = ['08/09','09/10','10/11','11/12','12/13','13/14','14/15','15/16','16/17','17/18','18/19']

In [59]:
def rankchances(i):
    df = read_df(i)
    playerlist = plist(df)
    totalchances = []
    throughballs = []
    crosses = []
    cutbacks = []
    others = []
    for p in playerlist:
        pdf=df.query("(type_name == 'Pass')&(player_name==@p)&\
                (pass_type_name not in ['Free Kick', 'Corner', 'Throw-in', 'Kick Off'])&\
                ((pass_shot_assist==True)or(pass_goal_assist==True))")
        tballs = len(pdf[pdf.pass_technique_name=='Through Ball'])
        crs = len(pdf[pdf.pass_cross==True])
        cbacks = len(pdf[pdf.pass_cut_back==True])
        oth = len(pdf) - tballs - crs - cbacks
        totalchances.append(len(pdf))
        throughballs.append(tballs)
        crosses.append(crs)
        cutbacks.append(cbacks)
        others.append(oth)
    chancedf = pd.DataFrame()
    chancedf['players'] = playerlist
    chancedf['open_play'] = totalchances
    chancedf['throughball'] = throughballs
    chancedf['cross'] = crosses
    chancedf['cutback'] = cutbacks
    chancedf['others'] = others
    chancedf = chancedf.sort_values(by=['open_play'],ascending=False)[:10].reset_index(drop=True).transpose()
    return chancedf

In [154]:
def waffleplotter(i):
    chancedf = rankchances(i)
    fig = plt.figure(FigureClass=Waffle,values=chancedf[0].tolist()[2:6],
                     rows=10,title={'label':chancedf[0].tolist()[0]+'\n'+
                                    'Season: '+season[i]+'\n'+
                                    " Total chances: "+str(chancedf[0].tolist()[1])},
                    legend = {'labels' : ["Throughballs","Crosses","Cutbacks","Others"],
                    'loc': 'upper left',
                    'bbox_to_anchor': (1.3, 1),
                    'fontsize': 8
            })
    filesavename = 'topchances'+str(i)+'.png'
    plt.savefig(filesavename)
    plt.close("all")

In [155]:
for i in range(11):
    waffleplotter(i)